# Lesson 1: Quick tour of Agents, LangSmith

It's very quick and simple to setup and run an agent. In this lab, we'll setup an agent, trace its trajectory in LangSmith and demonstrate Studio.

File Edit Options Buffers Tools Text Help                                                                                                                               
0/ Lesson 0 - intro w Harrison/Lance
1/ lesson 1 - [Slides only] overview on the common patterns of
   a. context isolation to sub agents,
   b. context offloading,
   c. todo list tool,
   d. task specific prompt eng.
      Context eng slides have many good citations for all. Plus the blog post. Should be quick.

   lesson 2 - build an agent fast with create_react_agent. Demo	LangSmith, Studio in simple environment.

2/ lesson 3 - todo tool / planning. Easy.

3/ lesson 4 - filesystem tool. Btw this is p cool and novel. It’s smtg we chatted abt. Mock filesystem in Langgraph state is nice shortcut.

4/ lesson 5 - subagent delegation. Also a bit novel how it does it; let’s supervisor define the subagent on the fly (tools, name, prompt). whereas for DR we carefully \
define subagent tools / system prompt, and only let supervisor define the research topic for each. This is more flexible, am curious on performance.

5/ lesson 6 - prompting + applying to task (like DR).  But ofc general is the story here.
Btw another little tip I’ve started doing is using a think_tool. Extremely useful. Claude interleaves thinking w/ tool calls. Can see it’s though process and understan\
d failures. Super helpful.



In [7]:
import os
from typing import Literal
from tavily import TavilyClient
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic
from langsmith_utils import print_trace_info

# Create the Tavily search tool
@tool
def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Run a web search using Tavily"""
    tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
    search_results = tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )
    return search_results

In [8]:
# Create the agent
def create_simple_search_agent():
    # Set up the model
    model = ChatAnthropic(model_name="claude-sonnet-4-20250514", max_tokens=64000)
    
    # Create the agent with the search tool
    agent = create_react_agent(
        model=model,
        tools=[internet_search],
        prompt="You are a helpful assistant that can search the internet. Use the internet_search tool to find information when needed."
    )
    
    return agent

In [9]:
# Run the example
if __name__ == "__main__":
    # Make sure you have TAVILY_API_KEY set
    if "TAVILY_API_KEY" not in os.environ:
        print("Please set TAVILY_API_KEY environment variable")
        exit(1)
    
    # Create the agent
    agent = create_simple_search_agent()
    
    # Run a search example
    print("Running search example...")
    result = agent.invoke({
        "messages": [{
            "role": "user", 
            "content": "What is the latest news about artificial intelligence?"
        }]
    })
    
    # Print the result
    print("\n=== Search Result ===")
    print(result["messages"][-1].content)


Running search example...

=== Search Result ===
Based on the latest search results, here are some key recent developments in artificial intelligence:

## Recent AI News Highlights:

**1. Google's Progress Toward AGI (August 5, 2025)**
- Google DeepMind has outlined their latest step toward creating Artificial General Intelligence (AGI)
- They've developed a new model that allows AI systems to interact with convincing simulations of the real world
- Google argues that "world models" are a crucial step toward achieving AGI - AI systems that can perform most tasks on par with humans

**2. OpenAI Corporate Developments (August 7, 2025)**
- OpenAI is reportedly about to make many employees millionaires due to the company's massive valuation growth
- The company, which started as a non-profit, is now valued like a major corporation
- OpenAI is offering ChatGPT Enterprise to federal agencies for just $1, showing their push into government contracts
- The company has also released two new ope

In [13]:
# Print the LangSmith projects URL for easy access to all traces
print(f"🎯 LangSmith Projects URL: https://smith.langchain.com/projects")



🎯 LangSmith Projects URL: https://smith.langchain.com/projects
